# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Colab Notebooks"

In [3]:
# %cd /content/gdrive/My Drive/Colab Notebooks

In [4]:
# import libraries
import numpy as np
import os
import datetime
import os
import keras
from skimage import io
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
from collections import Counter 
import cv2

Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30) #disabled for experimenting
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)
# tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())

In [7]:
# we started with batch size = 663 and slowly optimised it to fit into our resources 
batch_size = 30 #experiment with the batch size
frames = 30 #number of frames in each video 
# we start with 120*120 images and adjust these later for model building process
rows = 120 # final image height to be decided
cols = 120 # final image width to be decided
channels = 3
step_size = 1 #this will help in customising the frames chosen from the video for model building

In [8]:
# we distribute the images on the basis of size. This is just done for experiment puroposes later.
# outpaths = [train_path] #val_path
def image_segregation(path='train'):
    final_120 =[]
    final_360=[]
    doc = ''
    if(path=='train'):
        op = 'Project_data/train/'
        doc = train_doc
    else:
        op = 'Project_data/val/'
        doc = val_doc
    for f in doc:
        path = f.split(';')[0]
        imgs = os.listdir(op+path)
        for img in imgs:
            image = io.imread(op+path+'/'+img)
            if(image.shape[0]==360):
                final_120.append(op+path+'/'+img) 

            if(image.shape[0]==120):
                final_360.append(op+path+'/'+img)
    return (final_120, final_360)


In [9]:
# make list of images with corresponding sizes
# final_120, final_360 = image_segregation()

In [10]:
# view random imagees
#selectedIndexes  = []
#for i in range(3):
#    selectedIndexes.append(rn.randint(0,5520))


#s_im_120 = [final_120[i] for i in selectedIndexes]
#s_im_360 = [final_360[i] for i in selectedIndexes]
#print(s_im_120)
#print(s_im_360)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
# we have looked at random images from the set and decided the following dimensions for the images. This has helped to 
# remove the edges and preserve the center of the image where gesture data is present. 
def cropImage(image):
    (image_h, image_w, _) = image.shape
    print(image.shape)
    h_start=0
    h_end = image_h
    w_start=0
    w_end = image_w
    # for images 120*160
    if(image_h==120):
        h_start=20
        w_start=10
        w_end = 120
    # for images of size 360*360
    elif image_h==360:
        h_start=30
        w_start=30
        w_end = 320
    return image[h_start:h_end, w_start:w_end]

In [12]:
# this method will adjust the size of the image to given dimensions. The appropriate size will be found through experments.
def resize_image(image, height=rows, width=cols):
    return  resize(image, (height, width),anti_aliasing=False)

In [13]:
# we try different normalization technique
def normalize_image(image):
    norm_image = image - np.min(image)/np.max(image) - np.min(image)
#     norm_image = image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
    return norm_image

In [14]:
#utility to show processed images
def showImage(array, idx):
    a = array[idx]
    image = io.imread(a)
    plt.subplot(2, 2,1)
    plt.imshow(image)

    cropped = cropImage(image)
    plt.subplot(2,2,2)
    plt.imshow(cropped)

    resized = resize_image(cropped)
    plt.subplot(2, 2,3)
    plt.imshow(resized)

    
    normalized = normalize_image(resized)
    plt.subplot(2, 2,4)
    plt.imshow(normalized)

    plt.show()

In [15]:
#plt.figure(figsize=(300,300))
#showImage(s_im_360, 0)

In [16]:
# view random images
#plt.figure(figsize=(300,300))
#showImage(s_im_120, 0)

In [17]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [18]:
# Helper function to initialize all the batch image data and labels
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, frames, rows, cols, channels)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [19]:
# internal function to generate augmented data. 
# We have done 2 types of augmentation to prevent overfitting of data- flipping of the images and affine transformation. 
def fetch_aug_batchdata(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    
    # We will also build an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    
    # We will also build an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = init_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            ## image = imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (rows, cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [20]:
# wrapper generator function. it supports ablation experiment and a flag to decide between training and validation folder
def generator(source_path, folder_list, batch_size, validation=False,ablation=None):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    if(ablation!=None):
        folder_list=folder_list[:ablation]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield fetch_aug_batchdata(source_path, folder_list, batch, batch_size, t,validation)
            
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield fetch_aug_batchdata(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [21]:
# path to images folders
train_path = 'Project_data/train'
val_path = 'Project_data/val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)

num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

classes= 5 # output classes

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [22]:
# keras related imports
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, ZeroPadding3D
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

## LSTM Model

### Model 1

In [23]:
# we have decided to experiment with this optimizer first. So keeping it in common place
optimiser = optimizers.Adam(0.001) 

In [ ]:
feature_map=[8, 16, 32, 64, 128] # we will expirment with different number of features for different layers
dense_layer_size = [1000,500,5]
num_epochs = 10
batch_size = 10
input_shape = (frames, rows, cols, channels)

In [24]:
#write your model here
model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using LSTM as the RNN model along with softmax as our last layer.
model.add(LSTM(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

In [25]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 7, 7, 64)      0         
__________

In [26]:
# timestamp to use in model name. 
curr_dt_time = datetime.datetime.now()

model_name = 'model_init_lstm' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [27]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [28]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [29]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/10
23/23 [==============================] - 215s 9s/step - loss: 1.2777 - categorical_accuracy: 0.4541 - val_loss: 1.1535 - val_categorical_accuracy: 0.5700

Epoch 00001: val_loss improved from inf to 1.15351, saving model to model_init_lstm_2020-11-0709_28_02.025517/model-00001-1.27666-0.45450-1.15351-0.57000.h5
Epoch 2/10
23/23 [==============================] - 12s 501ms/step - loss: 1.2065 - categorical_accuracy: 0.5217 - val_loss: 2.0083 - val_categorical_accuracy: 0.3583

Epoch 00002: val_loss did not improve from 1.15351
Epoch 3/10
23/23 [==============================] - 16s 691ms/step - loss: 1.2278 - categorical_accuracy: 0.5459 - val_loss: 1.2475 - val_categorical_accuracy: 0.4333

Epoch 00003: val_loss did not improve from 1.15351
Epoch 4/10
23/23 [==============================] - 17s 722ms/step - loss: 1.4947 - categorical_accuracy: 0.2754 - val_loss: 1.4052 - val

### Model 2

#### We add batch normalization

In [ ]:
feature_map=[8, 16, 32, 64, 128] # we will expirment with different number of features for different layers
dense_layer_size = [1000,500,5]
num_epochs = 10
batch_size = 10
input_shape = (frames, rows, cols, channels)

In [30]:
#write your model here
model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))


model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using LSTM as the RNN model along with softmax as our last layer.
model.add(LSTM(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

In [31]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_10 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_11 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 7, 7, 64)      0         
__________

In [32]:
# timestamp to use in model name. 
curr_dt_time = datetime.datetime.now()

model_name = 'model_init_lstm' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [33]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [34]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [35]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = Source path =  Project_data/train ; batch size = 30
30
Epoch 1/10
23/23 [==============================] - 123s 5s/step - loss: 1.6733 - categorical_accuracy: 0.1853 - val_loss: 1.6663 - val_categorical_accuracy: 0.1867

Epoch 00001: val_loss improved from inf to 1.66629, saving model to model_init_lstm_2020-11-0709_34_02.147293/model-00001-1.66894-0.19256-1.66629-0.18667.h5
Epoch 2/10
23/23 [==============================] - 12s 529ms/step - loss: 1.6427 - categorical_accuracy: 0.1884 - val_loss: 1.5818 - val_categorical_accuracy: 0.2417

Epoch 00002: val_loss improved from 1.66629 to 1.58178, saving model to model_init_lstm_2020-11-0709_34_02.147293/model-00002-1.64273-0.18841-1.58178-0.24167.h5
Epoch 3/10
23/23 [==============================] - 15s 673ms/step - loss: 1.6063 - categorical_accuracy: 0.2754 - val_loss: 1.6405 - val_categorical_accuracy: 0.1917

Epoch 00003: val_loss did not improve from 1.58178
Epoch 4/10
23/23 [=========

### Model 4

#### Increase the learning rate

In [ ]:
feature_map=[8, 16, 32, 64, 128] 
dense_layer_size = [256,128,5] # we reduce the dense layer neurons
num_epochs = 10
batch_size = 10
input_shape = (frames, rows, cols, channels)

In [36]:

model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

# model.add(keras.layers.Dropout(0.4))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
# model.add(keras.layers.Dropout(0.4))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using LSTM as the RNN model along with softmax as our last layer.
model.add(LSTM(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

In [37]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_20 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_21 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_22 (TimeDis (None, 30, 30, 30, 16)    64        
_________________________________________________________________
time_distributed_23 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 30, 15, 15, 32)    128       
__________

In [38]:
# timestamp to use in model name. 
curr_dt_time = datetime.datetime.now()

model_name = 'model_init_lstm' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=2, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [39]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [41]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/20
23/23 [==============================] - 125s 5s/step - loss: 1.7123 - categorical_accuracy: 0.1964 - val_loss: 1.6057 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.60572, saving model to model_init_lstm_2020-11-0709_38_30.050657/model-00001-1.71878-0.19105-1.60572-0.21000.h5
Epoch 2/20
23/23 [==============================] - 13s 556ms/step - loss: 1.6829 - categorical_accuracy: 0.1981 - val_loss: 1.5462 - val_categorical_accuracy: 0.2833

Epoch 00002: val_loss improved from 1.60572 to 1.54617, saving model to model_init_lstm_2020-11-0709_38_30.050657/model-00002-1.68285-0.19807-1.54617-0.28333.h5
Epoch 3/20
23/23 [==============================] - 15s 654ms/step - loss: 1.6182 - categorical_accuracy: 0.2029 - val_loss: 1.5778 - val_categorical_accuracy: 0.2417

Epoch 00003: val_loss did not improve from 1.54617
Epoch 4/20
23/23 [=========

### Model 3

#### We reduce the dense layer neurons

In [ ]:
feature_map=[8, 16, 32, 64, 128] 
dense_layer_size = [256,128,5] # we reduce the dense layer neurons
num_epochs = 10
batch_size = 10
input_shape = (frames, rows, cols, channels)

In [36]:

model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

# model.add(keras.layers.Dropout(0.4))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
# model.add(keras.layers.Dropout(0.4))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))


model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using LSTM as the RNN model along with softmax as our last layer.
model.add(LSTM(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

In [37]:
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_20 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_21 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_22 (TimeDis (None, 30, 30, 30, 16)    64        
_________________________________________________________________
time_distributed_23 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_25 (TimeDis (None, 30, 15, 15, 32)    128       
__________

In [38]:
# timestamp to use in model name. 
curr_dt_time = datetime.datetime.now()

model_name = 'model_init_lstm' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [39]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [41]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/20
23/23 [==============================] - 125s 5s/step - loss: 1.7123 - categorical_accuracy: 0.1964 - val_loss: 1.6057 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.60572, saving model to model_init_lstm_2020-11-0709_38_30.050657/model-00001-1.71878-0.19105-1.60572-0.21000.h5
Epoch 2/20
23/23 [==============================] - 13s 556ms/step - loss: 1.6829 - categorical_accuracy: 0.1981 - val_loss: 1.5462 - val_categorical_accuracy: 0.2833

Epoch 00002: val_loss improved from 1.60572 to 1.54617, saving model to model_init_lstm_2020-11-0709_38_30.050657/model-00002-1.68285-0.19807-1.54617-0.28333.h5
Epoch 3/20
23/23 [==============================] - 15s 654ms/step - loss: 1.6182 - categorical_accuracy: 0.2029 - val_loss: 1.5778 - val_categorical_accuracy: 0.2417

Epoch 00003: val_loss did not improve from 1.54617
Epoch 4/20
23/23 [=========

## GRU MODEL

In [43]:
#write your model here
feature_map=[8, 16, 32, 64, 128, 256] # we will expirment with different number of features for different layers
dense_layer_size = [1000,500,5] # 
classes= 5
input_shape = (frames, rows, cols, channels)
model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Conv2D(feature_map[4], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [44]:
optimiser = optimizers.Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_30 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_31 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_32 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_33 (TimeDis (None, 30, 30, 30, 16)    64        
_________________________________________________________________
time_distributed_34 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_35 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_36 (TimeDis (None, 30, 15, 15, 32)    128       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [45]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [46]:
model_name = 'model_init_CNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [47]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [48]:
batch_size = 10
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/valSource path =  Project_data/train ; batch size = 30
Epoch 1/10
 ; batch size = 30
23/23 [==============================] - 125s 5s/step - loss: 1.3112 - categorical_accuracy: 0.4532 - val_loss: 1.0593 - val_categorical_accuracy: 0.6133

Epoch 00001: val_loss improved from inf to 1.05934, saving model to model_init_CNN_2020-11-0709_38_30.050657/model-00001-1.28882-0.46657-1.05934-0.61333.h5
Epoch 2/10
23/23 [==============================] - 10s 456ms/step - loss: 1.2311 - categorical_accuracy: 0.4783 - val_loss: 0.9083 - val_categorical_accuracy: 0.6250

Epoch 00002: val_loss improved from 1.05934 to 0.90826, saving model to model_init_CNN_2020-11-0709_38_30.050657/model-00002-1.23114-0.47826-0.90826-0.62500.h5
Epoch 3/10
23/23 [==============================] - 15s 651ms/step - loss: 1.6802 - categorical_accuracy: 0.3913 - val_loss: 1.4239 - val_categorical_accuracy: 0.3917

Epoch 00003: val_loss did not improve from 0.90826
Epoch 4/10
23/23 [===========

### Removing dropout layers

In [49]:
#write your model here
feature_map=[8, 16, 32, 64, 128, 256] # we will expirment with different number of features for different layers
dense_layer_size = [1000,500,5] # 
classes= 5
input_shape = (frames, rows, cols, channels)
model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[4], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

In [50]:
optimiser = optimizers.Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_44 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_45 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_46 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_47 (TimeDis (None, 30, 30, 30, 16)    64        
_________________________________________________________________
time_distributed_48 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_49 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_50 (TimeDis (None, 30, 15, 15, 32)    128       
__________

In [51]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [52]:
model_name = 'model_init_CNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [53]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [54]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train Epoch 1/20
; batch size = 10
67/67 [==============================] - 124s 2s/step - loss: 1.2078 - categorical_accuracy: 0.5205 - val_loss: 1.4374 - val_categorical_accuracy: 0.4833

Epoch 00001: val_loss improved from inf to 1.43737, saving model to model_init_CNN_2020-11-0709_38_30.050657/model-00001-1.21494-0.51785-1.43737-0.48333.h5
Epoch 2/20
67/67 [==============================] - 40s 592ms/step - loss: 1.2385 - categorical_accuracy: 0.5506 - val_loss: 1.3015 - val_categorical_accuracy: 0.5033

Epoch 00002: val_loss improved from 1.43737 to 1.30148, saving model to model_init_CNN_2020-11-0709_38_30.050657/model-00002-1.23845-0.55058-1.30148-0.50333.h5
Epoch 3/20
67/67 [==============================] - 44s 652ms/step - loss: 1.0254 - categorical_accuracy: 0.6053 - val_loss: 1.6796 - val_categorical_accuracy: 0.3400

Epoch 00003: val_loss did not improve from 1.30148
Epoch 4/20
67/67 [===========

### change image size to 84X84

In [55]:
#write your model here
feature_map=[8, 16, 32, 64, 128, 256] # we will expirment with different number of features for different layers
dense_layer_size = [1000,500,5] # 
classes= 5
rows = 84
cols = 84

input_shape = (frames, rows, cols, channels)

model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Conv2D(feature_map[4], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

In [56]:
optimiser = optimizers.Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_58 (TimeDis (None, 30, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_59 (TimeDis (None, 30, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_60 (TimeDis (None, 30, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_61 (TimeDis (None, 30, 21, 21, 16)    64        
_________________________________________________________________
time_distributed_62 (TimeDis (None, 30, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_63 (TimeDis (None, 30, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_64 (TimeDis (None, 30, 10, 10, 32)    128       
__________

In [57]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [58]:
model_name = 'model_init_CNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [59]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [60]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 96s 1s/step - loss: 1.3398 - categorical_accuracy: 0.4580 - val_loss: 1.1101 - val_categorical_accuracy: 0.6100

Epoch 00001: val_loss improved from inf to 1.11010, saving model to model_init_CNN_2020-11-0709_38_30.050657/model-00001-1.34024-0.45701-1.11010-0.61000.h5
Epoch 2/20
67/67 [==============================] - 31s 466ms/step - loss: 1.2500 - categorical_accuracy: 0.5423 - val_loss: 1.4793 - val_categorical_accuracy: 0.4300

Epoch 00002: val_loss did not improve from 1.11010
Epoch 3/20
67/67 [==============================] - 34s 512ms/step - loss: 1.1572 - categorical_accuracy: 0.5638 - val_loss: 1.1234 - val_categorical_accuracy: 0.5500

Epoch 00003: val_loss did not improve from 1.11010
Epoch 4/20
67/67 [==============================] - 36s 532ms/step - loss: 1.1537 - categorical_accuracy: 0.5638 - val_loss: 0.9708 - val_c

### Adding one more dense layer

In [61]:
 # reset image size to 120*120
rows = 120
cols = 120

In [62]:
#write your model here
feature_map=[8, 16, 32, 64, 128, 256] # we will expirment with different number of features for different layers
dense_layer_size = [1000, 500, 250, 5] # 
classes= 5

input_shape = (frames, rows, cols, channels)

model = Sequential()
# add multiple convulation layers
model.add(TimeDistributed(Conv2D(feature_map[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

model.add(TimeDistributed(Conv2D(feature_map[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(Conv2D(feature_map[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Conv2D(feature_map[4], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(dense_layer_size[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[1], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(dense_layer_size[2], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(classes, activation='softmax')) # using Softmax as last layer

In [63]:
optimiser = optimizers.Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_72 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_73 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_74 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_75 (TimeDis (None, 30, 30, 30, 16)    64        
_________________________________________________________________
time_distributed_76 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_77 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_78 (TimeDis (None, 30, 15, 15, 32)    128       
__________

In [64]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [65]:
model_name = 'model_init_CNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.001, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [66]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [67]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train Epoch 1/20
 Project_data/val ; batch size = 10
; batch size = 10
67/67 [==============================] - 123s 2s/step - loss: 1.2514 - categorical_accuracy: 0.4749 - val_loss: 1.2791 - val_categorical_accuracy: 0.6167

Epoch 00001: val_loss improved from inf to 1.27908, saving model to model_init_CNN_2020-11-0709_38_30.050657/model-00001-1.26011-0.47059-1.27908-0.61667.h5
Epoch 2/20
67/67 [==============================] - 41s 615ms/step - loss: 1.3051 - categorical_accuracy: 0.5456 - val_loss: 1.4970 - val_categorical_accuracy: 0.3667

Epoch 00002: val_loss did not improve from 1.27908
Epoch 3/20
67/67 [==============================] - 44s 653ms/step - loss: 1.2339 - categorical_accuracy: 0.4959 - val_loss: 1.2382 - val_categorical_accuracy: 0.4633

Epoch 00003: val_loss improved from 1.27908 to 1.23815, saving model to model_init_CNN_2020-11-0709_38_30.050657/model-00003-1.23393-0.49585-1.23815-0.46333.h5
Epoch 4/20
67/67 [===========

## Conv3D model

### Optimiser SGD

In [ ]:
filters = [8,16,32,64]
dense = [256, 128, 5]

# Input
input_shape=(frames, rows, cols, channels)

# Define model
model = Sequential()

model.add(Conv3D(filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(dense[2], activation='softmax'))

In [ ]:
optimiser = optimizers.SGD() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [ ]:
model_name = 'model_init_Conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 10
num_epochs = 10
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
 #                   callbacks=callbacks_list, validation_data=val_generator, 
  #                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

### Increased Epochs and Adam optimiser

In [ ]:
filters = [8,16,32,64]
dense = [256, 128, 5]

# Input
input_shape=(frames, rows, cols, channels)

# Define model
model = Sequential()

model.add(Conv3D(filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(dense[2], activation='softmax'))

In [ ]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [ ]:
model_name = 'model_init_Conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_loss', verbose=1, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 10
num_epochs = 20
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
 #                   callbacks=callbacks_list, validation_data=val_generator, 
  #                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

### Increasing dense perceptrons

In [ ]:
classes= 5
input_shape = (frames, rows, cols, channels)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 10
num_epochs = 20
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
 #                   callbacks=callbacks_list, validation_data=val_generator, 
  #                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

### Adding one extra layer with dropout

In [ ]:
classes= 5
input_shape = (frames, rows, cols, channels)

nb_filters = [8, 16, 32, 64, 128]
nb_dense = [1000, 500, 5]

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv3D(nb_filters[4], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))


#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 10
num_epochs = 20
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
 #                   callbacks=callbacks_list, validation_data=val_generator, 
  #                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

### Adding one more layer with BatchNormalization

In [ ]:
classes= 5
input_shape = (frames, rows, cols, channels)

nb_filters = [8, 16, 32, 64, 128, 256]
nb_dense = [1000, 500, 5]

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[4], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv3D(nb_filters[5], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))


#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 10
num_epochs = 20
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
 #                   callbacks=callbacks_list, validation_data=val_generator, 
  #                  validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)